In [ ]:
# !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Lg2jL89n3lqkKCulAnk4WwmI8G1hNfIA' -O BalancedNewsCorpusShuffled.zip
# !unzip BalancedNewsCorpusShuffled.zip

In [5]:
from torch.utils.data import Dataset
from sentence_transformers import SentenceTransformer
import pandas as pd
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
from tqdm.auto import tqdm
import re


def clean(text:str):
    not_used = re.compile('[^ .?!/@$%~|0-9|ㄱ-ㅣ가-힣]+')
    dup_space = re.compile('[ \t]+')  # white space duplicate
    dup_stop = re.compile('[\.]+')  # full stop duplicate
    cleaned = not_used.sub('', text.strip()) 
    cleaned = dup_space.sub(' ', cleaned)
    cleaned = dup_stop.sub('.', cleaned) 

    return cleaned

df = pd.read_csv('./BalancedNewsCorpusShuffled/BalancedNewsCorpus_train.csv')
df['News'] = df['News'].apply(lambda text: text.replace('<p>', '\n').replace('</p>', '\n'))
df['News'] = df['News'].apply(clean)
df

,filename,date,NewsPaper,Topic,News
0,NLRW1900000141,20170324,부산일보,스포츠,야구 종가 마침내 정상에 서다 야구 종가 미국이 푸에르토리코를 누르고 2017 월드...
1,NPRW1900000003,20110209,한국경제신문사,정치,외통위 27명중 15명 추가협상안만 처리 국회 외교통상통일위원회 소속 의원 27명 ...
2,NLRW1900000144,20100406,영남일보,사회,한나라 후보 희망연대 당원 구함 공천변수 작용 주목 오늘까지 추가 모집 오는 6월 ...
3,NLRW1900000064,20100804,광주매일신문,스포츠,모처럼 살아난 포 7타점 합작 12 3 4강 진입을 놓고 혈전을 벌이고 있는 가 와...
4,NLRW1900000070,20160615,광주매일신문,문화,문화전당서 동방의 등불 만나다 일찍이 아시아의 황금 시기에 빛나던 등불의 하나였던 ...
...,...,...,...,...,...
8995,NWRW1900000006,20141114,조선일보사,IT/과학,내가 구매한 영화 출근길의 오빠도 방에 있는 엄마도 보네? 스마트폰 가족 공유 콘텐...
8996,NIRW1900000022,20101217,노컷뉴스,연예,꽃보다 예쁜 터치 선웅의 여장 이게 바로 안산 4 수퍼 루키 터치의 멤버 선웅이 여...
8997,NLRW1900000092,20180131,국제신문,사회,어머니 빨리 쾌차하세요밀양참사 후 더 깊어진 고부애 같은 병실 입원해 극진히 수발 ...
8998,NLRW1900000103,20090206,대전일보,IT/과학,엑스포공원 드라마 타운 성공하려면 대전 엑스포 과학공원 내 조성될 것으로 기대를 모...


In [6]:
DATASET = 'BalancedNewsCorpus'  
MODEL = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'  
COLLECTION_NAME = 'BalancedNewsCorpus_db'  # Collection name
DIMENSION = 768  # Embeddings size
MILVUS_HOST = '127.0.0.1'
MILVUS_PORT = '19530' 
INDEX_TYPE = "IVF_FLAT"


connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
if connections:
    print("Milvus connected")
else:
    exit()

if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)
    
    
## 가지고 있는 샘플데이터의 dtype과 맞게, collection의 필드 정보들을 선언
EMBEDDING_FIELD_NAME ='News_embedding'
field2args = {
    'file_id': {'dtype': DataType.INT64, 'is_primary':True, 'auto_id':True},
    'filename': {'dtype': DataType.VARCHAR, 'max_length': 20},
    'date': {'dtype': DataType.INT64, 'max_length': 10},
    'NewsPaper': {'dtype': DataType.VARCHAR, 'max_length': 100},
    'Topic': {'dtype': DataType.VARCHAR, 'max_length': 100},
    'News': {'dtype': DataType.VARCHAR, 'max_length': 30000},
    EMBEDDING_FIELD_NAME: {'dtype': DataType.FLOAT_VECTOR, 'dim': DIMENSION},
}

# Milvus 연결을 설정합니다. 작업하기 전에 반드시 Milvus 서버에 연결해야 합니다.
connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)

if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

# 스키마 정의
fields = [
    FieldSchema(name = field_name, **args)
    for field_name, args in field2args.items()
]

schema = CollectionSchema(fields=fields)
collection = Collection(name=COLLECTION_NAME, schema=schema)

# collection에 이미 인덱스 유무 파악
if collection.has_index():
    try:
        collection.drop_index()
    except:
        collection.release()
        collection.drop_index()

# index 정의하기
index_params = {
    'metric_type': 'L2',         # "L2", "IP", "COSINE" 
    'index_type': INDEX_TYPE,    # "IVF_FLAT","IVF_SQ8", "IVF_PQ", "HNSW", "ANNOY"
    'params': {"nlist": 1},
}

collection.create_index(field_name=EMBEDDING_FIELD_NAME, index_params=index_params)
collection.load()
collection

Milvus connected


<Collection>:
-------------
<name>: BalancedNewsCorpus_db
<description>: 
<schema>: {'auto_id': True, 'description': '', 'fields': [{'name': 'file_id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'filename', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 20}}, {'name': 'date', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'NewsPaper', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}}, {'name': 'Topic', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}}, {'name': 'News', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 30000}}, {'name': 'News_embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'enable_dynamic_field': False}

In [7]:
embedder = SentenceTransformer(MODEL)
texts = df['News'].tolist()
embeddings = embedder.encode(texts, show_progress_bar=True, normalize_embeddings=True)

c:\Users\kidong\anaconda3\envs\milvus\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kidong\.cache\huggingface\hub\models--snunlp--KR-SBERT-V40K-klueNLI-augSTS. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Batches: 100%|██████████| 282/282 [59:59<00:00, 12.77s/it] 


In [25]:
embeddings.shape

(9000, 768)

In [8]:
# 만약에 DataFrame과 VectorDB에 적재할 Column이 다른 경우에 이와 같은 방식을 고려해볼 수 있음
column2field = {
    'filename':'filename',
    'date':'date',
    'NewsPaper':'NewsPaper',
    'Topic':'Topic',
    'News':'News',
}

field2column = {v:k for k,v in column2field.items()}


class CustomDataset(Dataset):
    
    def __init__(self, df, embeddings, field2column, embedding_field):
        self.df = df
        self.embeddings = embeddings
        self.field2column = field2column
        self.embedding_field = embedding_field
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        items = {field: self.df.iloc[idx][column] for field, column in self.field2column.items()}
        items[self.embedding_field] = self.embeddings[idx].tolist()
        return items
    
dataset = CustomDataset(df, embeddings, field2column, EMBEDDING_FIELD_NAME)

In [9]:
# insert 수행하기
for data in tqdm(iter(dataset), total=len(dataset)):
    collection.insert(data)
    
collection.flush()
collection.num_entities # 9000, 삽입된 엔티티의 수를 나타낸다.

100%|██████████| 9000/9000 [02:55<00:00, 51.36it/s]


9000

In [10]:
test_df = pd.read_csv('./BalancedNewsCorpusShuffled/BalancedNewsCorpus_test.csv')
test_df['News'] = test_df['News'].apply(lambda text: text.replace('<p>', '\n').replace('</p>', '\n'))
test_df['News'] = test_df['News'].apply(clean)
test_df

,filename,date,NewsPaper,Topic,News
0,NLRW1900000024,20120209,경기일보,IT/과학,인터넷 게임 족쇄에 도내 업체도 덜덜 업체 긍정적인 면 보지않는 일괄규제 답답 정부...
1,NLRW1900000029,20171018,경기일보,IT/과학,삼성전자 에 삼성만의 특화된 생태 보전 활동 자랑하다생물다양성 보존활동 우수 사례 ...
2,NLRW1900000029,20170920,경기일보,IT/과학,지자체의 카카오채널 도민들 관심 부족으로 폐쇄되거나 운영 부실 전국 지자체 중 최초...
3,NLRW1900000025,20130218,경기일보,IT/과학,돈만 잡아먹는 홈페이지 과감하게 로그아웃 경기도가 연간 152억여원의 비용을 들여 ...
4,NLRW1900000028,20160314,경기일보,IT/과학,콘텐츠기업 대상 부천 클러스터 입주사 모집 경기콘텐츠진흥원은 콘텐츠 기업 지원 공간...
...,...,...,...,...,...
1795,NPRW1900000004,20120903,한국경제신문사,정치,안철수 독자출마로 가나정치권 배제한채 대권 행보 안철수 서울대 융합과학기술대학원장이...
1796,NPRW1900000001,20091116,한국경제신문사,정치,예산심의 또 표류서민저소득층 울린다 국회의 예산심의가 표류하면서 자칫 경제에 먹구름...
1797,NPRW1900000005,20130221,한국경제신문사,정치,퇴직금 7000만원은 로비 대가 제기 커지는 김병관 의혹의 선택은 ? 박근혜 정부의...
1798,NPRW1900000004,20120706,한국경제신문사,정치,국회의원장관 겸직 금지하면 임명권자 인재풀 좁아져 반대 강창희 신임 국회의장은 6일...


In [11]:
test_idx = 123
query = test_df.iloc[test_idx]['News']
query

'카울리 클라우드 총괄 클라우드시장 금융서 폭발성장 물꼬만 터진다면 한국 클라우드 시장은 폭발적으로 성장할 것으로 예상합니다. 에서 글로벌 클라우드 시장을 총괄하고 있는 스티브 카울리 총괄 대표는 최근 매일경제신문과 인터뷰를 갖고 한국은 시장 규모에 비해 클라우드 도입 정도가 매우 낮다며 이같이 말했다. 카울리 대표는 예를 들어 금융산업 같은 경우 보안 이슈와 규제 문제 때문에 클라우드 도입을 주저하고 있는 게 사실이라면서도 처음 클라우드를 도입하는 은행은 여러 가지 이점을 선점할 수 있기 때문에 빠른 확산이 가능하다고 전망했다. 실제로 호주 벤디고은행은 톱4에 들지 못하는 하위 은행이었지만 클라우드블루믹스를 도입한 뒤 100년 넘은 은행이라는 전통적 이미지를 벗고 혁신 선도 은행으로 이미지를 새롭게 정립했다고 그는 설명했다. 한국 시장에 대한 낙관적 전망으로 은 지난해 퍼블릭 클라우드 데이터센터를 건설하기도 했다. 카울리 대표는 클라우드 서비스가 가격경쟁으로 가고 있지만 가격보다 가치라는 변수가 더 중요하다면서 한국에서 다수 잠재 고객사들을 만나고 난 뒤 새로운 클라우드 서비스를 도입하는 것은 어떤 가치를 제공해 주느냐의 싸움이라는 것을 더 확신하게 됐다고 강조했다. 특히 처럼 이미 한국에 비즈니스 고객들이 많았던 회사들이 클라우드 서비스를 제공한다면 수요자 니즈를 더 잘 파악할 수 있을 것이라고 했다. 카울리 대표는 은 인공지능 왓슨 블록체인 사물인터넷 데이터 애널리틱스 등 다양한 솔루션도 갖추고 있다고 부연했다. 카울리 대표는 앞으로 정부 시스템도 클라우드로 대체될 것이라는 흥미로운 전망도 했다. 그는 시민들이 더 많이 참여하는 정치 형태를 선호하면서 행정수요 처리 방식을 혁신할 필요성이 증가하고 있다며 시민과 정부 사이 관계를 개선하는 데 클라우드 서비스가 도움이 된다는 평가도 나오고 있다고 말했다. 카울리 대표는 세계적으로 보면 공공부문은 클라우드 사업의 주요한 시장이라며 해당 분야에 대한 연구를 강화하고 있다고 말했다. 은 최근 19분기 연속으로 전

In [12]:
query_embedding = embedder.encode(query, show_progress_bar=False, normalize_embeddings=True)
query_embedding.shape # (768,)

(768,)

In [13]:
result = collection.search(
    data = [query_embedding], # 앞서 생성한 동일한 크기의 임베딩 벡터 
    anns_field = EMBEDDING_FIELD_NAME, # 어떤 임베딩 필드를 기준으로 검색할 지 선언
    param = index_params, # 생성했었을 때의 동일한 index parameter
    limit = 5, # top K를 뜻하며 검색해서 보고 싶은 개수
    output_fields = ['filename','date', 'NewsPaper', 'Topic'] # 보고 싶은 필드 명시
)

In [14]:
result_list = []
for hits in result:
    for hit in hits:
        result_list.append(hit.to_dict())
result_list

[{'id': 451154865982057305,
  'distance': 0.5874919891357422,
  'entity': {'filename': 'NPRW1900000053',
   'date': 20110721,
   'NewsPaper': '매일경제신문사',
   'Topic': 'IT/과학'}},
 {'id': 451154865982043429,
  'distance': 0.6472843289375305,
  'entity': {'filename': 'NPRW1900000069',
   'date': 20170424,
   'NewsPaper': '이비뉴스',
   'Topic': 'IT/과학'}},
 {'id': 451154865982055591,
  'distance': 0.6537426710128784,
  'entity': {'filename': 'NIRW1900000022',
   'date': 20100810,
   'NewsPaper': '노컷뉴스',
   'Topic': 'IT/과학'}},
 {'id': 451154865982053137,
  'distance': 0.6564981341362,
  'entity': {'filename': 'NPRW1900000057',
   'date': 20150410,
   'NewsPaper': '매일경제신문사',
   'Topic': '경제'}},
 {'id': 451154865982054915,
  'distance': 0.6836014986038208,
  'entity': {'filename': 'NLRW1900000028',
   'date': 20160622,
   'NewsPaper': '경기일보',
   'Topic': 'IT/과학'}}]

In [23]:
result = collection.search(
        data = [query_embedding], # 앞서 생성한 동일한 크기의 임베딩 벡터 
        anns_field = EMBEDDING_FIELD_NAME, # 어떤 임베딩 필드를 기준으로 검색할 지 선언
        param = index_params, # 생성했었을 때의 동일한 index parameter
        expr = "NewsPaper in ['매일경제신문사', '이비뉴스'] and date > 20120101",
        limit = 5, # top K를 뜻하며 검색해서 보고 싶은 개수
        output_fields = ['filename','date', 'NewsPaper', 'Topic'] # 보고 싶은 필드 명시
    )

result_list = []
for hits in result:
    for hit in hits:
        result_list.append(hit.to_dict())
result_list

[{'id': 451154865982043429,
  'distance': 0.6472843289375305,
  'entity': {'filename': 'NPRW1900000069',
   'date': 20170424,
   'NewsPaper': '이비뉴스',
   'Topic': 'IT/과학'}},
 {'id': 451154865982053137,
  'distance': 0.6564981341362,
  'entity': {'filename': 'NPRW1900000057',
   'date': 20150410,
   'NewsPaper': '매일경제신문사',
   'Topic': '경제'}},
 {'id': 451154865982051349,
  'distance': 0.7227335572242737,
  'entity': {'filename': 'NPRW1900000057',
   'date': 20150318,
   'NewsPaper': '매일경제신문사',
   'Topic': 'IT/과학'}},
 {'id': 451154865982046561,
  'distance': 0.7266741991043091,
  'entity': {'filename': 'NPRW1900000068',
   'date': 20160128,
   'NewsPaper': '이비뉴스',
   'Topic': 'IT/과학'}},
 {'id': 451154865982051609,
  'distance': 0.7595013976097107,
  'entity': {'filename': 'NPRW1900000056',
   'date': 20140115,
   'NewsPaper': '매일경제신문사',
   'Topic': '생활'}}]

In [15]:
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def search(query, collection, model, embedding_field, output_fields, top_k=100, params={}):
    
    query_embedding = model.encode(query, show_progress_bar=False, normalize_embeddings=True)
    
    result = collection.search(
        data = [query_embedding],
        anns_field = embedding_field,
        param = params,
        limit = top_k,
        output_fields = output_fields
    )
    
    result_list = []
    for hits in result:
        for hit in hits:
            result_list.append(hit.to_dict())
    
    flatten_result = [flatten_dict(r) for r in result_list]
    result_df = pd.DataFrame.from_records(flatten_result)
    # result_df.insert(2, 'distance_abs', result_df['distance'].abs())
    
    # vector db의 id와 metric, 그리고 선언했던 output_fields에 대해 'entity_'라는 prefix 추가
    reorder_cols = ['id', 'distance',] + ['entity_'+f for f in output_fields]
    result_df = result_df.reindex(columns=reorder_cols)
    
    return result_df

In [17]:
result = search(
    query = query, 
    collection = collection, 
    model = embedder, 
    embedding_field = EMBEDDING_FIELD_NAME,
    output_fields = df.columns.tolist(),
    top_k = 5
)
result

,id,distance,entity_filename,entity_date,entity_NewsPaper,entity_Topic,entity_News
0,451154865982057305,0.587492,NPRW1900000053,20110721,매일경제신문사,IT/과학,보겔스 부사장 사용자 중심이 아마존의 미래 아마존에는 사용자의 모든 요구에 부응하려...
1,451154865982043429,0.647284,NPRW1900000069,20170424,이비뉴스,IT/과학,장화진 한국 대표 클라우드 국내 산업 접목 속도낸다 은 한국 시장에서 역사가 깊은 ...
2,451154865982055591,0.653743,NIRW1900000022,20100810,노컷뉴스,IT/과학,클라우드 서비스 기반 급 미디어 플랫폼 도입 전세계적으로 클라우드 컴퓨팅에 관한 ...
3,451154865982053137,0.656498,NPRW1900000057,20150410,매일경제신문사,경제,코드마니 퍼멀그룹 올해가 헤지펀드 투자 최적기 올해는 지난 2~3년과 달리 글로벌 ...
4,451154865982054915,0.683601,NLRW1900000028,20160622,경기일보,IT/과학,빅데이터 산업 이끄는 경기도 벤처기업 공간 인텔리전트 데이터 컨퍼런스 개최 경기도내...
